# hamburgermon + lumped resonator

In [6]:
import numpy as np
import numpy.linalg as alg

In [7]:
hbar = 1.054e-34
h = hbar*2*np.pi
qe = 1.609e-19
flux_quant = h/2/qe

# Unit conversions
MHz = 10.0**(-3)
GHz = 1.0
kHz = 10.0**(-6)
us = 10.0**3
ns = 1.0

In [8]:
# F
Crq = 13.4e-15 # res <-> pad1
Cr  = 146.5e-15 # res <-> gnd
Cj  = 19.6e-15 # pad1 <-> pad 2
C1  = 65.9e-15 # pad1 <-> gnd
C2  = 77.2e-15 # pad2 <-> gnd

res_freq = 6.7e9 # Hz

# H
Lr = 1/(2*np.pi * res_freq)**2/Cr
# print(Lr * 1e9, 'nH')
Lj = 5e-9

c_matrix = np.array([
    [Crq + Cr, -Crq, 0], 
    [-Crq, C1 + Cj + Crq, -Cj],
    [0, -Cj, C2 + Cj]
    ])

inv_l_matrix = np.array([
    [1/Lr, 0, 0],
    [0, 1/Lj, -1/Lj],
    [0, -1/Lj, 1/Lj]
])

print('Check symmetric:', np.allclose(c_matrix, c_matrix.T), np.allclose(l_matrix, l_matrix.T))

Check symmetric: True True


### simultaneous diagonalization

In [9]:
inv_c_matrix = alg.inv(c_matrix)
evals_c, u1 = alg.eig(inv_c_matrix)
# print(evals_c)
# print(u1)
u1 = np.array([u1[i]/np.sqrt(evals_c[i]) for i in range(len(evals_c))])
evals, evecs = alg.eig(u1.transpose() @ inv_l_matrix @ u1)

ECs = 1e-9 * qe**2/2/h * evecs.transpose() @ inv_c_matrix @ evecs
EJs = 1e-9 * flux_quant**2/h/4/np.pi**2 * evecs.transpose() @ inv_l_matrix @ evecs
Zs = np.array([np.sqrt(8*ECs[i,i]/EJs[i,i]) for i in range(len(evals_c))])
gs = np.array([[4*ECs[i,j]/np.sqrt(Zs[i]*Zs[j]) for i in range(len(evals_c))] for j in range(len(evals_c))])

print('Modes (cols are evecs)\n', evecs)
print('EC values (GHz)\n', ECs)
print('EJ values (GHz)\n', EJs)
print('impedances (unitless)\n', Zs)
print('couplings (GHz)\n', gs)



Modes (cols are evecs)
 [[-0.98976579  0.08599201  0.11388172]
 [ 0.0933194  -0.21371242  0.97242917]
 [-0.10795908 -0.9731045  -0.20350052]]
EC values (GHz)
 [[ 0.12213162  0.01177473 -0.01159349]
 [ 0.01177473  0.22604219 -0.03791878]
 [-0.01159349 -0.03791878  0.19435761]]
EJ values (GHz)
 [[42.51285498  1.37248493  2.92779156]
 [ 1.37248493 18.99417346 29.34303374]
 [ 2.92779156 29.34303374 45.34578217]]
impedances (unitless)
 [0.1515999  0.30855284 0.18517285]
couplings (GHz)
 [[ 3.22247233  0.21776931 -0.27678086]
 [ 0.21776931  2.93035307 -0.63454285]
 [-0.27678086 -0.63454285  4.19840392]]


In [10]:
res_mode = np.argmax(np.abs(evecs[0,:])) # look for column with max in row 0
print('res mode', res_mode)
assert np.abs(evecs[0,res_mode]) > 0.90, 'resonator mode is not the resonator mode??'
qubit_mode = -1
for i in range(len(evals_c)):
    if i == res_mode: continue
    sign1 = np.sign(evecs[i, i])
    j = (i+1)%len(evals_c)
    if j == res_mode: j = (j+1)%len(evals_c)
    sign2 = np.sign([evecs[j, i]])
    if sign1 * sign2 == -1: qubit_mode = i
if qubit_mode == -1: assert False, 'check it yourself'
print('qubit mode', qubit_mode)
print('EC', ECs[qubit_mode, qubit_mode])
print('EJ', EJs[qubit_mode, qubit_mode])
print('coupling to res', gs[res_mode, qubit_mode])
print('transmon freq', np.sqrt(8*ECs[qubit_mode,qubit_mode]*EJs[qubit_mode,qubit_mode]))

res mode 0
qubit mode 2
EC 0.19435760740377528
EJ 45.34578217475182
coupling to res -0.2767808626494465
transmon freq 8.396807835999358
